<a href="https://colab.research.google.com/github/rashida048/ApacheSpark-BigDataAnalytics/blob/main/Assignment3_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 198 kB 33.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=6093c69b2a72efa3aa6201a01fa93b6772a61281e8b3760eb7f69210921874e1
  Stored in directory: /root/.cache/pip/wheels/a5/0a/c1/9561f6fecb759579a7d863dcd846daaa95f598744e71b02c77
Successfully built pyspark


In [2]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [3]:
rd = sc.textFile("taxi-data-sorted-small.csv.bz2")

In [4]:
words = rd.map(lambda x: x.split(','))

In [5]:
def isfloat(value):
    try:
        float(value)
        return True
    except:
        return False

In [6]:
def correctRows(p):
    if(len(p)==17):
        if(isfloat(p[5]) and isfloat(p[11])):
            if(float(p[5]) !=0 and float(p[11]) !=0):
                return p

In [7]:
wordsCorr = words.filter(lambda x: correctRows(x))

In [8]:
def zeroOrOne(x):
  if (x >= 1 and x <= 6):
    return 1.0
  else:
    return 0.0

In [9]:
def date(x):
  return x[:10]

In [10]:
night_ride_date = wordsCorr.map(lambda x: x + [date(x[2])] + [zeroOrOne(int(x[2][-7]))])

In [11]:
night_ride_date.take(2)

[['07290D3599E7A0D62097A346EFCC1FB5',
  'E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01 00:00:00',
  '2013-01-01 00:02:00',
  '120',
  '0.44',
  '-73.956528',
  '40.716976',
  '-73.962440',
  '40.715008',
  'CSH',
  '3.50',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '4.50',
  '2013-01-01',
  0.0],
 ['0EC22AAF491A8BD91F279350C2B010FD',
  '778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01 00:01:00',
  '2013-01-01 00:03:00',
  '120',
  '0.71',
  '-73.973145',
  '40.752827',
  '-73.965897',
  '40.760445',
  'CSH',
  '4.00',
  '0.50',
  '0.50',
  '0.00',
  '0.00',
  '5.00',
  '2013-01-01',
  0.0]]

In [12]:
five_feat = night_ride_date.map(lambda x: (x[1], x[-2], float(x[4])/3600, float(x[5]), 1.0, float(x[15]), x[-1], float(x[11]) + float(x[14]) ))

In [13]:
five_feat.take(2)

[('E7750A37CAB07D0DFF0AF7E3573AC141',
  '2013-01-01',
  0.03333333333333333,
  0.44,
  1.0,
  0.0,
  0.0,
  3.5),
 ('778C92B26AE78A9EBDF96B49C67E4007',
  '2013-01-01',
  0.03333333333333333,
  0.71,
  1.0,
  0.0,
  0.0,
  4.0)]

In [14]:
type(five_feat)

pyspark.rdd.PipelinedRDD

In [15]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [16]:
df = five_feat.toDF(['driver_id', 'date', 'time', 'distance', 'no of ride', 'toll', 'No night ride', 'fare'])

In [ ]:
df.show()

+--------------------+----------+--------------------+--------+----------+----+-------------+----+
|           driver_id|      date|                time|distance|no of ride|toll|No night ride|fare|
+--------------------+----------+--------------------+--------+----------+----+-------------+----+
|E7750A37CAB07D0DF...|2013-01-01| 0.03333333333333333|    0.44|       1.0| 0.0|          0.0| 3.5|
|778C92B26AE78A9EB...|2013-01-01| 0.03333333333333333|    0.71|       1.0| 0.0|          0.0| 4.0|
|BE317B986700F63C4...|2013-01-01| 0.03333333333333333|    0.48|       1.0| 0.0|          0.0| 4.0|
|7077F9FD5AD649AEA...|2013-01-01| 0.03333333333333333|    0.61|       1.0| 0.0|          0.0| 4.0|
|CF8604E72D83840FB...|2013-01-01|0.016666666666666666|    0.39|       1.0| 0.0|          0.0| 3.7|
|BB899DFEA9CC964B5...|2013-01-01| 0.06666666666666667|    1.71|       1.0| 0.0|          0.0| 6.5|
|B7567F5BFD558C665...|2013-01-01| 0.06666666666666667|    1.21|       1.0| 0.0|          0.0| 7.3|
|ED3685521

In [17]:
data = df.groupby('driver_id', 'date').sum('time', 'distance', 'no of ride', 'toll', 'No night ride', 'fare')

In [20]:
rdd1 = data.rdd.map(list)

In [21]:
rdd1.take(2)

[['36C9437BD2FF31940BEBED44DDDDDB8A',
  '2013-01-01',
  4.516666666666666,
  79.96999999999998,
  26.0,
  0.0,
  17.0,
  326.28],
 ['60604EB4EBC0EEA81F5BF933621FAD16',
  '2013-01-01',
  6.383333333333334,
  104.32000000000001,
  26.0,
  4.8,
  14.0,
  422.24]]